Loading the model

In [1]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from transformers import pipeline
import torch

/home/epsysoev/.conda/envs/my_py_env1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
feature_extractor = pipeline("feature-extraction", framework="pt", model='llama2_model/llama2_model', device=0)
def get_embedding(text):
    return feature_extractor(text,return_tensors = "pt")[0].numpy().mean(axis=0)

Loading checkpoint shards: 100%|██████████| 6/6 [01:02<00:00, 10.38s/it]


In [6]:
torch.cuda.is_available()

True

In [4]:
#test if embeddings make sense
#good = get_embedding('No I am good')
#no_good = get_embedding('I am no good')
#print(good)
#print(no_good)
#print(good == no_good)

Getting embedding

In [2]:

overviews = pd.read_csv('processed_data/overviews.csv')

In [3]:
overviews.head()

,Unnamed: 0,movieId,original_title,overview,overview_and_title
0,0,1,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Toy Story:\nLed by Woody, Andy's toys live hap..."
1,1,2,Jumanji,When siblings Judy and Peter discover an encha...,Jumanji:\nWhen siblings Judy and Peter discove...
2,2,3,Grumpier Old Men,A family wedding reignites the ancient feud be...,Grumpier Old Men:\nA family wedding reignites ...
3,3,4,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Waiting to Exhale:\nCheated on, mistreated and..."
4,4,5,Father of the Bride Part II,Just when George Banks has recovered from his ...,Father of the Bride Part II:\nJust when George...


In [4]:
len(overviews)

44571

In [7]:
def data():
    for idx, row in overviews.iterrows():
        yield row['overview_and_title']


embeddings = []
for features in feature_extractor(data(), return_tensors = "pt"):
    features.shape
    embedding = features[0].numpy().mean(axis=0)
    embeddings.append(embedding)
    
overviews['vector'] = embeddings

overviews.to_pickle('processed_data/overviews_embedding.pkl')

In [8]:
tst = pd.read_pickle('processed_data/overviews_embedding.pkl')

In [9]:
overviews.head(2)['vector']

0    [0.23192838, -1.8712475, -0.47501668, 0.430822...
1    [1.2924119, -1.4130445, 0.38714024, 0.12504956...
Name: vector, dtype: object